<a href="https://colab.research.google.com/github/DonghaeSuh/PyTorch_Basic/blob/main/Binary_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 라이브러리 불러오기

In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

### Dataset Class

In [3]:
class CustomDataset(Dataset):
  def __init__(self,path):
    df=pd.read_csv(path)
    self.x1=df.iloc[:,0].values
    self.x2=df.iloc[:,1].values
    self.x3=df.iloc[:,2].values
    self.y=df.iloc[:,3].values
    self.length=len(df)

  def __getitm__(self,index):
    x=torch.FloatTensor([self.x1[index],self.x2[index],self.x3[index]])
    y=torch.FloatTensor([self.y[index]])
    return x,y

  def __len__(self):
    return self.length